In [1]:
!wget -nc "http://www.fega.es/sites/default/files/files/Beneficiaries_municipality_financial_year_2014.txt"

File 'Beneficiaries_municipality_financial_year_2014.txt' already there; not retrieving.



In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('Beneficiaries_municipality_financial_year_2014.txt', delimiter=';', encoding='latin1')
df.head()

,BENEFICIARIO,PROVINCIA,MUNICIPIO,MEDIDA,IMPORTE_EUROS
0,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.1 Single payment scheme,"17105,53"
1,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.7 Specific support,"2830,77"
2,A CABADA SC,Lugo,27016 - Chantada,I.1 Single payment scheme,"9416,07"
3,A CABADA SC,Lugo,27016 - Chantada,I.7 Specific support,4241
4,A CABADA SC,Lugo,27016 - Chantada,V/B.2.4 Agri-environment payments [Med.214],"9538,68"


In [4]:
df['amount'] = pd.to_numeric(df['IMPORTE_EUROS'].str.replace(',', '.'))

In [5]:
df['recipient_postcode'] = df['MUNICIPIO'].str.split(' - ').apply(lambda x: x [0])
df.head()

,BENEFICIARIO,PROVINCIA,MUNICIPIO,MEDIDA,IMPORTE_EUROS,amount,recipient_postcode
0,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.1 Single payment scheme,"17105,53",17105.53,15074
1,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.7 Specific support,"2830,77",2830.77,15074
2,A CABADA SC,Lugo,27016 - Chantada,I.1 Single payment scheme,"9416,07",9416.07,27016
3,A CABADA SC,Lugo,27016 - Chantada,I.7 Specific support,4241,4241.00,27016
4,A CABADA SC,Lugo,27016 - Chantada,V/B.2.4 Agri-environment payments [Med.214],"9538,68",9538.68,27016


In [6]:
df['recipient_location'] = df.apply(lambda x: '%s, %s' % (x['MUNICIPIO'].split(' - ')[1], x['PROVINCIA']), 1)
df.head()

,BENEFICIARIO,PROVINCIA,MUNICIPIO,MEDIDA,IMPORTE_EUROS,amount,recipient_postcode,recipient_location
0,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.1 Single payment scheme,"17105,53",17105.53,15074,"Rois, A Coruña"
1,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.7 Specific support,"2830,77",2830.77,15074,"Rois, A Coruña"
2,A CABADA SC,Lugo,27016 - Chantada,I.1 Single payment scheme,"9416,07",9416.07,27016,"Chantada, Lugo"
3,A CABADA SC,Lugo,27016 - Chantada,I.7 Specific support,4241,4241.00,27016,"Chantada, Lugo"
4,A CABADA SC,Lugo,27016 - Chantada,V/B.2.4 Agri-environment payments [Med.214],"9538,68",9538.68,27016,"Chantada, Lugo"


In [7]:
from slugify import slugify

df = df.rename(columns={
    'BENEFICIARIO': 'recipient_name',
    'MEDIDA': 'scheme',
})
df = df.drop(['PROVINCIA', 'MUNICIPIO', 'IMPORTE_EUROS'], 1)
df['country'] = 'ES'
df['currency'] = 'EUR'
df['recipient_id'] = df.apply(lambda x: 'ES-%s-%s' % (x['recipient_postcode'], slugify(x['recipient_name'])), 1)
df.head()

,recipient_name,scheme,amount,recipient_postcode,recipient_location,country,currency,recipient_id
0,A BOUCIÑA S.C.,I.1 Single payment scheme,17105.53,15074,"Rois, A Coruña",ES,EUR,ES-15074-a-boucina-s-c
1,A BOUCIÑA S.C.,I.7 Specific support,2830.77,15074,"Rois, A Coruña",ES,EUR,ES-15074-a-boucina-s-c
2,A CABADA SC,I.1 Single payment scheme,9416.07,27016,"Chantada, Lugo",ES,EUR,ES-27016-a-cabada-sc
3,A CABADA SC,I.7 Specific support,4241.00,27016,"Chantada, Lugo",ES,EUR,ES-27016-a-cabada-sc
4,A CABADA SC,V/B.2.4 Agri-environment payments [Med.214],9538.68,27016,"Chantada, Lugo",ES,EUR,ES-27016-a-cabada-sc


In [8]:
df['year'] = 2014

In [9]:
df.to_csv('es_2014.csv', index=False, encoding='utf-8')